In [1]:
import numpy as np
import math
from scipy.signal import convolve2d

# Data Generation

In [2]:
rows = 7
cols = 7
mu, sigma = 0, math.sqrt(2.0 / (rows * cols))

k_size = 3

filter2d = np.random.normal(mu, sigma, size=(k_size, k_size))

image = np.random.randint(255, size=(rows, cols))

print("filter = ", filter2d)
print("image = ", image)

filter =  [[-0.09637199  0.10539893  0.14333563]
 [-0.05828793 -0.31783076 -0.01237399]
 [-0.2408099   0.75504788  0.12997741]]
image =  [[139  98 115  53 253  28 118]
 [209 247  51 123  29 228  21]
 [129  98  76  29 189  16  91]
 [ 67 173 112  92  10 194  94]
 [241  59 107 145 236  99 237]
 [ 84   4 206 214 143 135 220]
 [201 164 246  94 137  60 170]]


## Scipy Convolution

In [3]:
scipy_res = convolve2d(image, filter2d, mode='same', boundary='fill', fillvalue=0)
print("scipy_res = ", scipy_res)

scipy_res =  [[ -51.66619688   11.50519869  -11.92695843  -15.53548881  -83.98502811
     7.25601987   -2.9566013 ]
 [   4.68054342    1.80048637   79.63262899  -51.63880145  189.67967603
   -28.43128256   95.12397316]
 [  42.00194949  181.25109728   41.66962675   96.12888689  -82.83781247
   170.95685643   54.08488392]
 [  62.13852767   40.56881845   25.79825169  -37.07162797  163.11284851
   -31.10763559   77.6821228 ]
 [ -62.6400848    76.99926323   43.36915406   58.74966268  -77.04586138
    90.46719676   62.17727756]
 [ 146.20799816   58.17604664   15.9213172    19.62348976  139.36202243
     0.43016251  146.73881015]
 [ -10.98242204 -104.61879518   18.8316848   113.01423583   55.07443039
    36.86602337  128.88381482]]


## My implementation

In [4]:
def conv2d(x, f):
    r, c = x.shape
    kr, kc = f.shape
    res = np.zeros(x.shape)
    for i in range(r):
        for j in range(c):
            # (i, j) is the center position of filter
            for ki in range(-int(kr / 2), int(kr / 2) + 1, 1):
                for kj in range(-int(kc / 2), int(kc / 2) + 1, 1):
                    m = i - ki
                    n = j - kj
                    #print(ki, kj, i, j)
                    if m >= 0 and m < r and n >= 0 and n < c:
                        res[i, j] += x[m, n] * f[ki + int(kr / 2), kj + int(kc / 2)]
    return res
res = conv2d(image, filter2d)
print("my_res = ", res)

my_res =  [[ -51.66619688   11.50519869  -11.92695843  -15.53548881  -83.98502811
     7.25601987   -2.9566013 ]
 [   4.68054342    1.80048637   79.63262899  -51.63880145  189.67967603
   -28.43128256   95.12397316]
 [  42.00194949  181.25109728   41.66962675   96.12888689  -82.83781247
   170.95685643   54.08488392]
 [  62.13852767   40.56881845   25.79825169  -37.07162797  163.11284851
   -31.10763559   77.6821228 ]
 [ -62.6400848    76.99926323   43.36915406   58.74966268  -77.04586138
    90.46719676   62.17727756]
 [ 146.20799816   58.17604664   15.9213172    19.62348976  139.36202243
     0.43016251  146.73881015]
 [ -10.98242204 -104.61879518   18.8316848   113.01423583   55.07443039
    36.86602337  128.88381482]]
